In [ ]:
import os
import cv2
import base64
import requests
import numpy as np
import mediapipe as mp
os.makedirs("_output", exist_ok=True)

In [ ]:
def expand_polygon(ls_keypoints, scale_factor):
    # Convert keypoints to a NumPy array
    points = np.array(ls_keypoints, dtype=np.float32)
    # Calculate the centroid of the polygon
    centroid = np.mean(points, axis=0)
    # Translate points to origin (subtract centroid), scale, and translate back
    expanded_points = centroid + (points - centroid) * scale_factor
    # Convert back to integer coordinates for OpenCV
    expanded_points = expanded_points.astype(np.int32)
    return expanded_points.tolist()

def get_keypoints(img_path, img_debug_path, expand_scale_factor):
    # Read image
    img_raw = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
    h, w, _ = img_raw.shape
    # MediaPipe: Extract face landmarks
    with mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(img_rgb)
        if results.multi_face_landmarks:
            for facelandmarks in results.multi_face_landmarks:
                # Landmark indices
                ls_indices = [10,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172,58,132,93,234,127,162,21,54,103,67,109]
                # Get coordinates
                ls_keypoints = []
                for idx in ls_indices:
                    ls_keypoints.append(
                        (int(facelandmarks.landmark[idx].x * w), int(facelandmarks.landmark[idx].y * h)) # A keypoint (x,y)
                    )
                # Expand polygon
                ls_keypoints_expanded = expand_polygon(ls_keypoints, expand_scale_factor)
                # Draw points
                cv2.polylines(img_raw, [np.array(ls_keypoints, dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=5)
                cv2.polylines(img_raw, [np.array(ls_keypoints_expanded, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=5)
                for keypoint in ls_keypoints:
                    cv2.circle(img_raw, keypoint, 5, (0, 255, 0), -1)
                cv2.imwrite(img_debug_path, img_raw)
                # Return
                return ls_keypoints_expanded, h, w
    return None # No faces detected

def create_mask(ls_keypoints, img_h, img_w, img_mask_path):
    # Convert to numpy array (required format for OpenCV fillPoly)
    ls_keypoints = np.array(ls_keypoints, np.int32)
    ls_keypoints = ls_keypoints.reshape((-1, 1, 2)) # shape (n,1,2)
    # Create a blank black image (size depends on your image)
    img_mask = np.zeros((img_h, img_w, 3), dtype=np.uint8)
    # Fill the polygon with white
    cv2.fillPoly(img_mask, [ls_keypoints], (255, 255, 255))
    cv2.imwrite(img_mask_path, img_mask)

def imagen_head_replace(img_path, msk_path, out_path, sd_params, instance_url):

    with open(img_path, "rb") as f_img, open(msk_path, "rb") as f_msk:
        img_base64 = base64.b64encode(f_img.read()).decode('utf-8')
        msk_base64 = base64.b64encode(f_msk.read()).decode('utf-8')
    img_h, img_w, _ = cv2.imread(img_path).shape

    payload = {
        "prompt": sd_params["prompt"],
        "negative_prompt": "",
        "styles": [],
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": "Euler a",
        "scheduler": "Automatic",
        "batch_size": 1,
        "n_iter": 1,
        "steps": sd_params["steps"],
        "cfg_scale": sd_params["cfg"],
        "distilled_cfg_scale": 3.5,
        "width": img_w,
        "height": img_h,
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "eta": 0,
        "denoising_strength": sd_params["denoising"],
        "s_min_uncond": 0.0,
        "s_churn": 0.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "s_noise": 1.0,
        "override_settings": {"sd_model_checkpoint": "MJX_V07", "CLIP_stop_at_last_layers": 1},
        "override_settings_restore_afterwards": False,
        # "refiner_checkpoint": "string",
        "refiner_switch_at": 0,
        "disable_extra_networks": False,
        # "firstpass_image": "string",
        "comments": {},
        "init_images": [img_base64],
        "resize_mode": 0,
        "image_cfg_scale": 1.5,
        "mask": msk_base64,
        "mask_blur_x": 4,
        "mask_blur_y": 4,
        "mask_blur": 4,
        "mask_round": True,
        "inpainting_fill": 1,
        "inpaint_full_res": 0,
        "inpaint_full_res_padding": 32,
        "inpainting_mask_invert": 0,
        "initial_noise_multiplier": 1.0,
        # "latent_mask": "string",
        # "force_task_id": "string",
        "hr_distilled_cfg": 3.5,
        "sampler_index": "Euler",
        "include_init_images": False,
        "script_name": None,
        "script_args": [],
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
        # "infotext": "string"
    }

    def make_request_to_forge_api(payload, savepath, instance_url):
        try:
            with requests.post(url=instance_url, json=payload) as req:
                with open(savepath, "wb") as f:
                    f.write(base64.b64decode(req.json()["images"][0]))
        except Exception as e:
            print(f"⚠️ Error > {e}")
            print(f"{req.json()}")

    make_request_to_forge_api(payload, out_path, instance_url)

In [ ]:
# Variables
img_path = r"D:\DRIVE\Drive\ig_xouuuus\IG_Girls_Photos\xouuuus\3.jpg"
instance_url = "https://4cr9e6b875v1nr-1234.proxy.runpod.net/sdapi/v1/img2img"
# Constants
msk_path = "_output/img_mask.jpg"
out_path = "_output/output.jpg"

sd_params = {
    "prompt": "best quality, masterpiece, ultra high res, (photorealistic:1.4), 1girl, <lora:LORA_KoreaDL:0.7>",
    "cfg": 7,
    "denoising": 0.7,
    "steps": 25,
}

ls_keypoints, img_h, img_w = get_keypoints(img_path, "_output/img_keypoints.jpg", expand_scale_factor=2.1)
create_mask(ls_keypoints, img_h, img_w, msk_path)
imagen_head_replace(img_path, msk_path, out_path, sd_params, instance_url)

In [ ]:
# TODO: compare with payloadexample.json to remove unused param in the payload